# Работа с API

API – это механизмы, которые позволяют двум программным компонентам взаимодействовать друг с другом, используя набор определений и протоколов. Например, система ПО метеослужбы содержит ежедневные данные о погоде. Приложение погоды на телефоне «общается» с этой системой через API и показывает ежедневные обновления погоды на телефоне.

С помощью API можно легко собрать нужную информацию из сайта или приложения.

# На примере HH

In [ ]:
# !pip install -U requests

In [1]:
import requests

import pandas as pd

In [2]:
url = 'https://api.hh.ru/vacancies'

text = 'аналитик данных'

In [3]:
parameters = {
    'vacancy_search_fields': 'name',
    'text': text,
    'area': '113',
    'per_page': '10',
    'page': 1,
    'responses_count_enabled': True,
}

In [4]:
r = requests.get(url, params=parameters)
r

<Response [200]>

In [5]:
vacancies_info = r.json()

In [6]:
df = pd.DataFrame({
    'id':[],
    'name':[],
    'salary':[],
    'company':[],
    'url': []
})

In [7]:
for vac in vacancies_info['items']:
    vac_parser = {
        'id': vac['id'],
        'name': vac['name'],
        'salary': vac['salary'],
        'company': vac['employer']['name'],
        'url': vac['url']
    }
    df.loc[len(df)] = vac_parser

In [8]:
df

,id,name,salary,company,url
0,88284712,Финансовый аналитик,"{'from': None, 'to': 220000, 'currency': 'RUR'...",SberTech,https://api.hh.ru/vacancies/88284712?host=hh.ru
1,86324076,Разработчик Python (junior),"{'from': None, 'to': 80000, 'currency': 'RUR',...",ANABAR,https://api.hh.ru/vacancies/86324076?host=hh.ru
2,88031308,Junior Data Analyst,None,VipDeposits,https://api.hh.ru/vacancies/88031308?host=hh.ru
3,88334720,Гейм-дизайнер,None,Ритейл-Процессинг,https://api.hh.ru/vacancies/88334720?host=hh.ru
4,87239633,QA engineer,None,"Иннотех, Группа компаний",https://api.hh.ru/vacancies/87239633?host=hh.ru
5,88186907,Экономист-аналитик (junior data scientist),None,Аналитический центр при Правительстве Российск...,https://api.hh.ru/vacancies/88186907?host=hh.ru
6,88233548,Стажер-Аналитик / Junior Analyst,None,Caltat,https://api.hh.ru/vacancies/88233548?host=hh.ru
7,87614461,Координатор,"{'from': 153200, 'to': None, 'currency': 'RUR'...",SberTech,https://api.hh.ru/vacancies/87614461?host=hh.ru
8,88281262,Начальник склада,"{'from': 120000, 'to': None, 'currency': 'RUR'...",ТД Ломоносовский фарфор,https://api.hh.ru/vacancies/88281262?host=hh.ru
9,86583145,Специалист по социальной работе,"{'from': 91000, 'to': None, 'currency': 'RUR',...",ГБУЗ Центр паллиативной помощи ДЗМ,https://api.hh.ru/vacancies/86583145?host=hh.ru


# Парсинг с помощью BeautifulSoup

Beautiful Soup - это библиотека для Python, которая позволяет парсить (анализировать) HTML и XML документы. Она предоставляет удобный способ искать, навигировать, и модифицировать дерево DOM (Document Object Model), представляющее HTML/XML документ.

In [22]:
# !pip install -U beautifulsoup4

In [9]:
from bs4 import BeautifulSoup

In [10]:
url = 'http://quotes.toscrape.com/'

In [11]:
r = requests.get(url, verify=False, timeout=10)
r

<Response [200]>

In [12]:
soup = BeautifulSoup(r.content, 'html.parser')

In [13]:
divs = soup.find_all('div', attrs={'class':'quote'})

In [14]:
df = pd.DataFrame({
    'text':[],
    'author':[],
    'author_url':[],
    'tags':[],
})

In [15]:
def get_quotes(soup, data=df):
    
    divs = soup.find_all('div', attrs={'class':'quote'})
    
    for div in divs:
        quote = {}
        quote['text'] = div.find('span', class_='text').text
        quote['author'] = div.find('small', class_='author').text
        quote['author_url'] = div.find_all('span')[1].find('a').get('href')
        tags = []
        for a_tag in div.find('div', class_='tags').find_all('a', class_='tag'):
            tags.append(a_tag.text)
        quote['tags'] = ', '.join(tags)
        df.loc[len(df)] = quote

In [16]:
url = 'https://quotes.toscrape.com/page/'

for i in range(1, 21):
    print(f'Page N:{i}')
    r = requests.get(url + str(i), timeout=10)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    get_quotes(soup, df)
    
    if 'No quotes found!' in soup.text:
        break

Page N:1
Page N:2
Page N:3
Page N:4
Page N:5
Page N:6
Page N:7
Page N:8
Page N:9
Page N:10
Page N:11


In [17]:
df.tail()

,text,author,author_url,tags
95,“You never really understand a person until yo...,Harper Lee,/author/Harper-Lee,better-life-empathy
96,“You have to write the book that wants to be w...,Madeleine L'Engle,/author/Madeleine-LEngle,"books, children, difficult, grown-ups, write, ..."
97,“Never tell the truth to people who are not wo...,Mark Twain,/author/Mark-Twain,truth
98,"“A person's a person, no matter how small.”",Dr. Seuss,/author/Dr-Seuss,inspirational
99,“... a mind needs books as a sword needs a whe...,George R.R. Martin,/author/George-R-R-Martin,"books, mind"


In [66]:
df

,text,author,author_url,tags
0,“The world as we have created it is a process ...,Albert Einstein,/author/Albert-Einstein,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,/author/J-K-Rowling,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,/author/Albert-Einstein,"inspirational, life, live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,/author/Jane-Austen,"aliteracy, books, classic, humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,/author/Marilyn-Monroe,"be-yourself, inspirational"
...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,/author/Harper-Lee,better-life-empathy
96,“You have to write the book that wants to be w...,Madeleine L'Engle,/author/Madeleine-LEngle,"books, children, difficult, grown-ups, write, ..."
97,“Never tell the truth to people who are not wo...,Mark Twain,/author/Mark-Twain,truth
98,"“A person's a person, no matter how small.”",Dr. Seuss,/author/Dr-Seuss,inspirational
